In [7]:
#ONE VS ALL

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Función de sigmoide
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Función para entrenar un modelo binario
def train_logistic_regression(X, y, learning_rate, num_iterations):
    m, n = X.shape
    theta = np.zeros(n)

    for iteration in range(num_iterations):
        z = X.dot(theta)
        h = sigmoid(z)
        gradient = X.T.dot(h - y) / m
        theta -= learning_rate * gradient

    return theta

# Carga el conjunto de datos Iris como ejemplo
iris = load_iris()
X = iris.data
y = iris.target

# Divide el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# Número de clases
num_classes = len(np.unique(y))

# Inicializa una lista vacía para almacenar los clasificadores binarios
binary_classifiers = []

# Entrena un clasificador binario para cada clase
for i in range(num_classes):
    y_binary = (y_train == i).astype(int)
    theta = train_logistic_regression(X_train, y_binary, learning_rate=0.1, num_iterations=1000)
    binary_classifiers.append(theta)

# Realiza predicciones para cada clasificador binario
predictions = np.array([sigmoid(X_test.dot(theta)) for theta in binary_classifiers])

# Encuentra la clase con la probabilidad más alta para cada instancia
predicted_classes = np.argmax(predictions, axis=0)

# Imprime manualmente las características, las etiquetas reales y las etiquetas predichas

# Calcula la precisión
accuracy = np.mean(predicted_classes == y_test)
print(f"Precisión: {accuracy:.2f}")

# Imprime la lógica de votación
for i in range(len(X_test)):
    votes = {class_id: 0 for class_id in range(num_classes)}

    for j in range(num_classes):
        if predicted_classes[i] == j:
            votes[j] += 1
        else:
            votes[np.argmax(predictions[:, j])] += 1

    print(f"Instancia {i}: Votos {votes}, Predicción: {predicted_classes[i]}, Valor Real: {y_test[i]}")

Precisión: 0.90
Instancia 0: Votos {0: 1, 1: 0, 2: 2}, Predicción: 0, Valor Real: 0
Instancia 1: Votos {0: 1, 1: 0, 2: 2}, Predicción: 2, Valor Real: 1
Instancia 2: Votos {0: 1, 1: 0, 2: 2}, Predicción: 2, Valor Real: 2
Instancia 3: Votos {0: 1, 1: 0, 2: 2}, Predicción: 2, Valor Real: 2
Instancia 4: Votos {0: 1, 1: 1, 2: 1}, Predicción: 1, Valor Real: 1
Instancia 5: Votos {0: 1, 1: 0, 2: 2}, Predicción: 2, Valor Real: 2
Instancia 6: Votos {0: 1, 1: 1, 2: 1}, Predicción: 1, Valor Real: 1
Instancia 7: Votos {0: 1, 1: 1, 2: 1}, Predicción: 1, Valor Real: 1
Instancia 8: Votos {0: 1, 1: 0, 2: 2}, Predicción: 2, Valor Real: 1
Instancia 9: Votos {0: 1, 1: 0, 2: 2}, Predicción: 0, Valor Real: 0
Instancia 10: Votos {0: 1, 1: 1, 2: 1}, Predicción: 1, Valor Real: 1
Instancia 11: Votos {0: 1, 1: 0, 2: 2}, Predicción: 0, Valor Real: 0
Instancia 12: Votos {0: 1, 1: 0, 2: 2}, Predicción: 0, Valor Real: 0
Instancia 13: Votos {0: 1, 1: 0, 2: 2}, Predicción: 2, Valor Real: 2
Instancia 14: Votos {0: 1, 1

In [8]:
#ONE VS ONE

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Función de sigmoide
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Función para entrenar un modelo binario
def train_logistic_regression(X, y, learning_rate, num_iterations):
    m, n = X.shape
    theta = np.zeros(n)

    for iteration in range(num_iterations):
        z = X.dot(theta)
        h = sigmoid(z)
        gradient = X.T.dot(h - y) / m
        theta -= learning_rate * gradient

    return theta

# Carga el conjunto de datos Iris como ejemplo
iris = load_iris()
X = iris.data
y = iris.target

# Divide el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# Número de clases
num_classes = len(np.unique(y))

# Inicializa una lista de matrices para almacenar los clasificadores binarios
binary_classifiers = [np.zeros((X.shape[1],)) for _ in range(num_classes)]

# Entrena un clasificador binario para cada par de clases
for i in range(num_classes):
    for j in range(i+1, num_classes):
        # Filtra los datos para las clases actuales
        X_train_ij = X_train[(y_train == i) | (y_train == j)]
        y_train_ij = y_train[(y_train == i) | (y_train == j)]
        
        # Crea etiquetas binarias para las clases actuales
        y_binary = (y_train_ij == i).astype(int)
        
        # Entrena un clasificador binario para las clases actuales
        theta = train_logistic_regression(X_train_ij, y_binary, learning_rate=0.1, num_iterations=1000)
        
        # Almacena el modelo binario en la lista de clasificadores binarios
        binary_classifiers[i] = theta

# Realiza predicciones para cada par de clases
num_instances = X_test.shape[0]
predictions = np.zeros((num_instances, num_classes))

for i in range(num_classes):
    for j in range(i+1, num_classes):
        theta = binary_classifiers[i]
        probability = sigmoid(X_test.dot(theta))
        predictions[:, i] += probability
        predictions[:, j] += (1 - probability)

# Encuentra la clase con la probabilidad más alta para cada instancia
predicted_classes = np.argmax(predictions, axis=1)

# Calcula la precisión
accuracy = np.mean(predicted_classes == y_test)
print(f"Precisión: {accuracy:.2f}")

# Imprime los votos para cada instancia
for i in range(num_instances):
    votes = {class_id: 0 for class_id in range(num_classes)}

    for j in range(num_classes):
        for k in range(j+1, num_classes):
            if predicted_classes[i] == j:
                votes[j] += 1
            else:
                votes[k] += 1

    print(f"Instancia {i}: Votos {votes}, Predicción: {predicted_classes[i]}, Valor Real: {y_test[i]}")
    

Precisión: 0.97
Instancia 0: Votos {0: 2, 1: 0, 2: 1}, Predicción: 0, Valor Real: 0
Instancia 1: Votos {0: 0, 1: 2, 2: 1}, Predicción: 1, Valor Real: 1
Instancia 2: Votos {0: 0, 1: 1, 2: 2}, Predicción: 2, Valor Real: 2
Instancia 3: Votos {0: 0, 1: 1, 2: 2}, Predicción: 2, Valor Real: 2
Instancia 4: Votos {0: 0, 1: 2, 2: 1}, Predicción: 1, Valor Real: 1
Instancia 5: Votos {0: 0, 1: 1, 2: 2}, Predicción: 2, Valor Real: 2
Instancia 6: Votos {0: 0, 1: 2, 2: 1}, Predicción: 1, Valor Real: 1
Instancia 7: Votos {0: 0, 1: 2, 2: 1}, Predicción: 1, Valor Real: 1
Instancia 8: Votos {0: 0, 1: 1, 2: 2}, Predicción: 2, Valor Real: 1
Instancia 9: Votos {0: 2, 1: 0, 2: 1}, Predicción: 0, Valor Real: 0
Instancia 10: Votos {0: 0, 1: 2, 2: 1}, Predicción: 1, Valor Real: 1
Instancia 11: Votos {0: 2, 1: 0, 2: 1}, Predicción: 0, Valor Real: 0
Instancia 12: Votos {0: 2, 1: 0, 2: 1}, Predicción: 0, Valor Real: 0
Instancia 13: Votos {0: 0, 1: 1, 2: 2}, Predicción: 2, Valor Real: 2
Instancia 14: Votos {0: 0, 1

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionReset

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionReset

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionReset

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionReset

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionReset

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionReset

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionReset

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\flore\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\flore\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionReset